In [5]:
import pandas as pd
import numpy as np
import yfinance as yf

tickers = ['MMM', 'SHOP', 'SAIA', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADM', 'ADBE']
momentum_returns_df = pd.DataFrame()

for ticker in tickers:
    tickerData = yf.Ticker(ticker)
    tickerDf = tickerData.history(period='1d', start='2020-01-01', end='2023-11-16')

    tickerDf['Pct_Change'] = tickerDf['Close'].pct_change() * 100

    if tickerDf['Pct_Change'].abs().max() > 15:
        continue  # Skip this ticker if any day's move is greater than 15%
    tickerDf.dropna(inplace=True)  

    # Calculate 6-month momentum
    tickerDf['6M_Momentum'] = tickerDf['Close'].pct_change(periods=6*30)  # Assuming ~30 days per month

    # Get the latest 6-month momentum value
    latest_momentum = tickerDf['6M_Momentum'].iloc[-1]

    # Create a temporary DataFrame and append it
    temp_df = pd.DataFrame({'Ticker': [ticker], '6M_Momentum': [latest_momentum]})
    momentum_returns_df = pd.concat([momentum_returns_df, temp_df], ignore_index=True)

# Rank stocks by their momentum
momentum_returns_df.sort_values(by='6M_Momentum', ascending=False, inplace=True)


In [6]:
backtest_start_date = '2020-07-01'
backtest_end_date = '2021-01-01'

# DataFrame to store backtesting results
backtest_results = pd.DataFrame()

for ticker in tickers:
    # Get historical data for the backtesting period
    tickerData = yf.Ticker(ticker)
    backtest_data = tickerData.history(period='1d', start=backtest_start_date, end=backtest_end_date)

    # Calculate performance over the 6-month backtesting period
    if not backtest_data.empty:
        start_price = backtest_data['Close'].iloc[0]
        end_price = backtest_data['Close'].iloc[-1]
        performance = (end_price - start_price) / start_price

        # Add the results to the DataFrame
        backtest_results = backtest_results.append({'Ticker': ticker, 'Performance': performance}, ignore_index=True)


AttributeError: 'DataFrame' object has no attribute 'append'

In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm

tickers = ['MMM', 'SHOP', 'SAIA', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADM', 'ADBE']
momentum_return_data = pd.DataFrame()

for ticker in tickers:
    tickerData = yf.Ticker(ticker)

    # Historical data for the first half of 2022
    historical_data = tickerData.history(start='2022-01-01', end='2022-06-30')
    # Calculate 6-month momentum
    historical_data['6M_Momentum'] = historical_data['Close'].pct_change(periods=6*30)
    print(historical_data)

    # Data for the second half of 2022
    future_data = tickerData.history(start='2022-07-01', end='2022-12-31')
    print(future_data)
    if not future_data.empty and not historical_data.empty:
        future_return = (future_data['Close'].iloc[-1] - future_data['Close'].iloc[0]) / future_data['Close'].iloc[0]
        latest_momentum = historical_data['6M_Momentum'].iloc[-1]

        # Create a DataFrame for the current ticker
        temp_df = pd.DataFrame({'Ticker': [ticker],
                                '6M_Momentum': [latest_momentum],
                                'Future_Return': [future_return]})
        
        # Concatenate with the main DataFrame
        momentum_return_data = pd.concat([momentum_return_data, temp_df], ignore_index=True)

# Drop any rows with NaN values
momentum_return_data.dropna(inplace=True)

if not momentum_return_data.empty:
    # Linear regression
    X = momentum_return_data['6M_Momentum']
    y = momentum_return_data['Future_Return']
    X = sm.add_constant(X)  # Adds a constant term to the predictor

    # Fit the model
    model = sm.OLS(y, X).fit()
    r_squared = model.rsquared

    print(f"R-squared value: {r_squared}")
else:
    print("Insufficient data for regression analysis.")


Insufficient data for regression analysis.
